In [ ]:
import os
current_working_directory = os.getcwd()

# hokey datasett:

In [5]:
import os
import shutil

def restructure_videos(input_path):
    # Define the target directories for Fight and NonFight
    fight_dir = os.path.join(input_path, 'Fight')
    nonfight_dir = os.path.join(input_path, 'NonFight')

    # Create the directories if they do not exist
    if not os.path.exists(fight_dir):
        os.makedirs(fight_dir)

    if not os.path.exists(nonfight_dir):
        os.makedirs(nonfight_dir)

    # Iterate over files in the input directory
    for file_name in os.listdir(input_path):
        file_path = os.path.join(input_path, file_name)

        # Skip directories
        if os.path.isdir(file_path):
            continue

        # Check if the file name starts with "fi" (Fight) or "no" (NonFight)
        if file_name.lower().startswith('fi'):
            # Move to 'fight' directory
            shutil.move(file_path, os.path.join(fight_dir, file_name))
            print(f"Moved {file_name} to 'fight' directory")
        elif file_name.lower().startswith('no'):
            # Move to 'nonfight' directory
            shutil.move(file_path, os.path.join(nonfight_dir, file_name))
            print(f"Moved {file_name} to 'nonfight' directory")
        else:
            print(f"Skipping {file_name} (does not match 'fi' or 'no')")

# Example usage
input_path = os.path.join(current_working_directory, "archive", "hockey")
restructure_videos(input_path)


Moved fi100_xvid.avi to 'fight' directory
Moved fi101_xvid.avi to 'fight' directory
Moved fi102_xvid.avi to 'fight' directory
Moved fi103_xvid.avi to 'fight' directory
Moved fi104_xvid.avi to 'fight' directory
Moved fi105_xvid.avi to 'fight' directory
Moved fi106_xvid.avi to 'fight' directory
Moved fi107_xvid.avi to 'fight' directory
Moved fi108_xvid.avi to 'fight' directory
Moved fi109_xvid.avi to 'fight' directory
Moved fi10_xvid.avi to 'fight' directory
Moved fi110_xvid.avi to 'fight' directory
Moved fi111_xvid.avi to 'fight' directory
Moved fi112_xvid.avi to 'fight' directory
Moved fi113_xvid.avi to 'fight' directory
Moved fi114_xvid.avi to 'fight' directory
Moved fi115_xvid.avi to 'fight' directory
Moved fi116_xvid.avi to 'fight' directory
Moved fi117_xvid.avi to 'fight' directory
Moved fi118_xvid.avi to 'fight' directory
Moved fi119_xvid.avi to 'fight' directory
Moved fi11_xvid.avi to 'fight' directory
Moved fi120_xvid.avi to 'fight' directory
Moved fi121_xvid.avi to 'fight' dire

# Airtby datasett

In [19]:
import os
import shutil
import uuid

def restructure_dataset(input_dir, output_dir):
    """
    Restructure dataset to match the desired format with unique IDs.
    
    Parameters:
        input_dir (str): Path to the input dataset directory.
        output_dir (str): Path to the output dataset directory.
    """
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Create subdirectories for "Fight" and "NonFight"
    fight_dir = os.path.join(output_dir, "Fight")
    non_fight_dir = os.path.join(output_dir, "NonFight")
    os.makedirs(fight_dir, exist_ok=True)
    os.makedirs(non_fight_dir, exist_ok=True)

    # Process "violent" and "non-violent" directories
    for category in ["violent", "non-violent"]:
        category_path = os.path.join(input_dir, category)

        if not os.path.exists(category_path):
            print(f"Category directory not found: {category_path}")
            continue

        # Determine target directory
        target_dir = fight_dir if category == "violent" else non_fight_dir

        # Process subdirectories (e.g., "cam1", "cam2")
        for subdir in os.listdir(category_path):
            subdir_path = os.path.join(category_path, subdir)

            if not os.path.isdir(subdir_path):
                continue

            # Process video files in each subdirectory
            for filename in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, filename)

                if not os.path.isfile(file_path):
                    continue

                # Generate a unique ID for each video
                unique_id = str(uuid.uuid4()) + os.path.splitext(filename)[1]

                # Copy the file to the target directory
                target_file_path = os.path.join(target_dir, unique_id)
                shutil.copy(file_path, target_file_path)

    print(f"Dataset restructured successfully to: {output_dir}")


# Example usage
input_dataset_path = os.path.join(current_working_directory, "archive", "violence-detection-dataset")
output_dataset_path = os.path.join(current_working_directory, "archive", "airtby")
restructure_dataset(input_dataset_path, output_dataset_path)


Dataset restructured successfully to: c:\Users\gorme\projects\godseye\apps\backend\dataset_processing\archive\airtby
